In [ ]:
# SETUP AUTOMATICO E DOWNLOAD DEL PROGETTO DA GITHUB
import os

# 1. Clona il repository (inserisci il tuo vero link GitHub)
!git clone https://github.com/TUO_NOME_UTENTE/AI-Studio.git

# 2. Sposta l'ambiente di lavoro dentro la cartella del progetto
%cd AI-Studio

# 3. Installa le librerie leggendo il file requirements.txt
!pip install -r requirements.txt
!apt-get install -y ffmpeg

print("✅ Ambiente clonato e pronto all'uso! Tutto il codice è sincronizzato.")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
import os
import shutil

# Metadati base
api_key_input = widgets.Password(description="API Key:", style={'description_width': 'initial'})
materia_input = widgets.Text(description="Materia:")
argomento_input = widgets.Text(description="Argomento:")
professore_input = widgets.Text(description="Professore:")

print("⚙️ COMPILA I METADATI E INSERISCI LA CHIAVE API:")
display(api_key_input, materia_input, argomento_input, professore_input)

# === SELEZIONE TIPO DI FONTI ===
print("\n📋 SELEZIONA LE FONTI DA UTILIZZARE:")
use_video = widgets.Checkbox(value=True, description="📹 Videolezione")
use_slides = widgets.Checkbox(value=False, description="📊 Slide del corso (PDF)")
use_notes = widgets.Checkbox(value=False, description="📝 Appunti di altri studenti (PDF)")

display(widgets.HBox([use_video, use_slides, use_notes]))

# Contenitore per upload dinamici
upload_area = widgets.VBox()

# Variabili globali per tracciare i file
video_filename = None
video_url = None
cookie_filename = None
slides_filenames = []
notes_filenames = []
history_filename = None

def update_upload_fields(change):
    """Mostra upload in base alle fonti selezionate"""
    widgets_list = []
    
    # VIDEOLEZIONE
    if use_video.value:
        widgets_list.append(widgets.HTML("<h3>🔗 OPZIONE A: Scarica da URL (Portale Universitario)</h3>"))
        global video_url_input
        video_url_input = widgets.Text(
            description="Link Video:", 
            placeholder="Incolla l'URL qui...", 
            layout=widgets.Layout(width='80%')
        )
        widgets_list.append(video_url_input)
        
        widgets_list.append(widgets.HTML("<h3>📤 OPZIONE B: Carica file MP4 dal PC</h3>"))
        btn_upload_video = widgets.Button(description="📹 Carica Video", button_style='info')
        
        def on_upload_video(b):
            global video_filename, video_upload
            print("Seleziona il file video...")
            video_upload = files.upload()
            if video_upload:
                uploaded_name = list(video_upload.keys())[0]
                os.makedirs("data/videos", exist_ok=True)
                video_filename = os.path.join("data/videos", uploaded_name)
                shutil.move(uploaded_name, video_filename)
                print(f"✅ Video salvato: {video_filename}")
        
        btn_upload_video.on_click(on_upload_video)
        widgets_list.append(btn_upload_video)
        
        widgets_list.append(widgets.HTML("<h3>🍪 (Opzionale) Cookies per portale protetto:</h3>"))
        btn_upload_cookies = widgets.Button(description="🍪 Carica cookies.txt", button_style='warning')
        
        def on_upload_cookies(b):
            global cookie_filename, cookie_upload
            cookie_upload = files.upload()
            if cookie_upload:
                cookie_filename = list(cookie_upload.keys())[0]
                print(f"✅ Cookies caricati: {cookie_filename}")
        
        btn_upload_cookies.on_click(on_upload_cookies)
        widgets_list.append(btn_upload_cookies)
    
    # SLIDE PDF
    if use_slides.value:
        widgets_list.append(widgets.HTML("<h3>📊 SLIDE DEL CORSO (PDF)</h3>"))
        btn_upload_slides = widgets.Button(description="📊 Carica Slide PDF", button_style='primary')
        
        def on_upload_slides(b):
            global slides_filenames, slides_upload
            slides_upload = files.upload()
            os.makedirs("data/documents", exist_ok=True)
            slides_filenames = []
            for name in slides_upload.keys():
                dest_path = os.path.join("data/documents", name)
                shutil.move(name, dest_path)
                slides_filenames.append(dest_path)
                print(f"✅ Slide salvate: {name}")
        
        btn_upload_slides.on_click(on_upload_slides)
        widgets_list.append(btn_upload_slides)
    
    # APPUNTI PDF
    if use_notes.value:
        widgets_list.append(widgets.HTML("<h3>📝 APPUNTI DI ALTRI STUDENTI (PDF)</h3>"))
        btn_upload_notes = widgets.Button(description="📝 Carica Appunti PDF", button_style='success')
        
        def on_upload_notes(b):
            global notes_filenames, notes_upload
            notes_upload = files.upload()
            os.makedirs("data/documents", exist_ok=True)
            notes_filenames = []
            for name in notes_upload.keys():
                dest_path = os.path.join("data/documents", name)
                shutil.move(name, dest_path)
                notes_filenames.append(dest_path)
                print(f"✅ Appunti salvati: {name}")
        
        btn_upload_notes.on_click(on_upload_notes)
        widgets_list.append(btn_upload_notes)
    
    # APPUNTI PRECEDENTI (sempre visibile)
    widgets_list.append(widgets.HTML("<h3>📚 (OPZIONALE) APPUNTI PRECEDENTI (.tex per continuità):</h3>"))
    btn_upload_history = widgets.Button(description="📚 Carica .tex precedenti", button_style='warning')
    
    def on_upload_history(b):
        global history_filename, history_upload
        history_upload = files.upload()
        if history_upload:
            history_filename = list(history_upload.keys())[0]
            print(f"✅ Appunti precedenti caricati: {history_filename}")
    
    btn_upload_history.on_click(on_upload_history)
    widgets_list.append(btn_upload_history)
    
    # Aggiorna il contenuto del VBox
    upload_area.children = widgets_list

# Osserva i cambiamenti nelle checkbox
use_video.observe(update_upload_fields, names='value')
use_slides.observe(update_upload_fields, names='value')
use_notes.observe(update_upload_fields, names='value')

# Mostra i campi iniziali
update_upload_fields(None)
display(upload_area)

In [ ]:
import sys
import os
import warnings

# Silenzia TUTTI i warning (console pulita)
warnings.filterwarnings('ignore')
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'

# VERIFICA PREREQUISITI
if not api_key_input.value:
    raise SystemExit("❌ STOP: Inserisci l'API Key nella cella precedente, poi riesegui questa cella.")

# Verifica che almeno una fonte sia stata selezionata
if not any([use_video.value, use_slides.value, use_notes.value]):
    raise SystemExit("❌ STOP: Seleziona almeno una fonte (video, slide o appunti) nella cella precedente.")

# Configura API Key PRIMA degli import
os.environ['GOOGLE_API_KEY'] = api_key_input.value

sys.path.append('./src')

# Importa moduli
from audio_processing import process_video_audio
from vision_processing import extract_keyframes
from llm_gen_engine import generate_notes
from document_processing import process_documents, merge_sources
from google.colab import files
import yt_dlp

# === PIPELINE CONDIZIONALE BASATA SULLE FONTI ===

video_transcript = None
video_keyframes = None
pdf_chunks = None

# PROCESSING VIDEOLEZIONE
if use_video.value:
    # Download da URL se fornito
    if video_url_input.value:
        video_name = f"{materia_input.value}_{argomento_input.value}".replace(" ", "_") or "lezione"
        video_filename = os.path.join("data/videos", f"{video_name}.mp4")
        
        ydl_opts = {
            'outtmpl': video_filename,
            'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        }
        if cookie_filename:
            ydl_opts['cookiefile'] = cookie_filename
            
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url_input.value])
    
    if not video_filename:
        raise SystemExit("❌ STOP: Nessun video fornito. Carica un video o inserisci un URL.")
    
    print("🎙️ Fase 1: Trascrizione audio...")
    video_transcript, transcript_path = process_video_audio(
        video_filename, 
        save_transcript=True,
        transcript_output_dir="data/transcripts"
    )
    
    print("👁️ Fase 2: Estrazione keyframe...")
    video_keyframes = extract_keyframes(video_filename, output_dir="data/frames")

# PROCESSING DOCUMENTI PDF
if use_slides.value or use_notes.value:
    all_pdfs = []
    pdf_types = []
    
    if use_slides.value and slides_filenames:
        all_pdfs.extend(slides_filenames)
        pdf_types.extend(["slides"] * len(slides_filenames))
        print(f"📊 Processing {len(slides_filenames)} slide PDF...")
    
    if use_notes.value and notes_filenames:
        all_pdfs.extend(notes_filenames)
        pdf_types.extend(["notes"] * len(notes_filenames))
        print(f"📝 Processing {len(notes_filenames)} appunti PDF...")
    
    if all_pdfs:
        pdf_chunks = process_documents(all_pdfs, pdf_types)

# MERGE DELLE FONTI
print("\n🔄 Unificazione delle fonti...")
unified_chunks, source_description, input_mode = merge_sources(
    video_transcript=video_transcript,
    video_keyframes=video_keyframes,
    pdf_chunks=pdf_chunks
)

print(f"📚 Fonti utilizzate: {source_description}")
print(f"📦 Chunks totali da processare: {len(unified_chunks)}")

# GENERAZIONE LATEX
print("\n🧠 Fase 3: Generazione LaTeX...")
latex_output = generate_notes(
    transcript=unified_chunks,  # Chunks unificati
    keyframes=[],  # Già inclusi nei chunks
    history_path=history_filename or "",
    input_mode=input_mode  # Passa modalità per selezione prompt automatica
)

nome_file = f"{materia_input.value}_{argomento_input.value}_{professore_input.value}".replace(" ", "_") + ".tex"

with open(nome_file, "w", encoding="utf-8") as f:
    f.write(latex_output)
    
files.download(nome_file)

print(f"\n✅ Completato! File: {nome_file}")

In [ ]:
from interactive_agent import run_editor_agent, run_tutor_agent
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Trascrizione unificata per contesto (gestisce assenza video)
if video_transcript:
    full_transcript = " ".join([seg['text'] for seg in video_transcript])
else:
    # Se non c'è video, estrai testo dai chunks PDF
    full_transcript = " ".join([chunk.get('text', '') for chunk in (pdf_chunks or [])])

# Widget
mode_selector = widgets.Dropdown(
    options=['Modalità TUTOR (Domande)', 'Modalità EDITOR (Modifica LaTeX)'],
    description='Azione:',
    style={'description_width': 'initial'}
)
text_input = widgets.Textarea(description="Richiesta:", layout=widgets.Layout(width='80%', height='100px'))
latex_input = widgets.Textarea(description="Codice LaTeX:", layout=widgets.Layout(width='80%', height='150px'))
submit_button = widgets.Button(description="Invia all'IA", button_style='success')
output_area = widgets.Output()
input_area = widgets.VBox()

def update_input_fields(change):
    """Mostra solo i campi necessari in base alla modalità"""
    if "TUTOR" in mode_selector.value:
        input_area.children = [text_input]
    else:
        input_area.children = [latex_input, text_input]

def on_submit(b):
    with output_area:
        clear_output()
        print("Elaborazione in corso...")
        if "TUTOR" in mode_selector.value:
            risposta = run_tutor_agent(text_input.value, full_transcript)
            display(HTML(f"<b>👨‍🏫 Il Prof risponde:</b><br><pre style='white-space: pre-wrap;'>{risposta}</pre>"))
        else:
            nuovo_latex = run_editor_agent(latex_input.value, text_input.value, full_transcript)
            display(HTML(f"<b>📝 Nuovo LaTeX (copia e incolla nel file):</b><br><pre>{nuovo_latex}</pre>"))

mode_selector.observe(update_input_fields, names='value')
submit_button.on_click(on_submit)

# Inizializza con la modalità corrente
update_input_fields(None)

display(mode_selector, input_area, submit_button, output_area)